<a href="https://colab.research.google.com/github/thai94/d2l/blob/main/3.linear-neural-networks/3_3_concise_implementation_of_linear_regression_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [97]:
import numpy as np
from torch.utils import data
import torch

In [98]:
def synthetic_data(w, b, num_examples):
    """Generate y = Xw + b + noise."""
    X = torch.normal(0, 1, (num_examples, len(w)))
    y = torch.matmul(X, w) + b
    y += torch.normal(0, 0.01, y.shape)
    return X, y.reshape((-1, 1))

In [99]:
true_w = torch.tensor([2, -3.4])
true_b = 4.2

features, labels = synthetic_data(true_w, true_b, 1000)

In [100]:
def load_array(data_arrays, batch_size, is_train=True):
  dataset = data.TensorDataset(*data_arrays)
  return data.DataLoader(dataset, batch_size, shuffle=is_train)

In [101]:
batch_size = 10
data_iter = load_array((features, labels), batch_size)

In [102]:
next(iter(data_iter))

[tensor([[ 0.9870,  0.3789],
         [ 0.3850, -1.2896],
         [ 0.3451, -1.0070],
         [-2.2304, -1.2944],
         [ 0.3541,  1.1616],
         [-0.2637, -0.9178],
         [ 0.9650, -0.1420],
         [-0.2868, -0.3212],
         [-0.3638,  0.6065],
         [-1.5730, -1.2857]]), tensor([[4.8795],
         [9.3461],
         [8.3233],
         [4.1373],
         [0.9494],
         [6.7777],
         [6.6184],
         [4.7188],
         [1.4086],
         [5.4145]])]

In [103]:
from torch import nn
net = nn.Sequential(nn.Linear(2, 1))

In [104]:
net[0].weight.data.normal_(0, 0.01)
net[0].bias.data.fill_(0)

tensor([0.])

In [105]:
net[0].weight

Parameter containing:
tensor([[-0.0044, -0.0184]], requires_grad=True)

In [106]:
net[0].bias

Parameter containing:
tensor([0.], requires_grad=True)

In [107]:
loss = nn.MSELoss()

In [108]:
trainer = torch.optim.SGD(net.parameters(), lr=0.03)

In [109]:
num_epochs = 5
for epoch in range(num_epochs):
  for X, y in data_iter:
    l = loss(net(X), y)
    trainer.zero_grad()
    l.backward()
    trainer.step()
  l = loss(net(features), labels)
  print(f'epoch {epoch + 1}, loss {l:f}')

epoch 1, loss 0.000321
epoch 2, loss 0.000106
epoch 3, loss 0.000106
epoch 4, loss 0.000105
epoch 5, loss 0.000106


In [110]:
w = net[0].weight.data
print('error in estimating w:', true_w - w.reshape(true_w.shape))
b = net[0].bias.data
print('error in estimating b:', true_b - b)

error in estimating w: tensor([ 7.5459e-05, -6.3944e-04])
error in estimating b: tensor([0.0007])
